In [4]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install plotly.express

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import plotly.express as px
from datetime import datetime, date
dt_string = datetime.now().strftime('%y%m%d')
pd.set_option('display.max_columns', 500)

In [7]:
cd_name = "221020-1111-cf11_cd_21-output"
cd_cf11 = pd.read_excel(f'input/cierres_f11/{cd_name}.xlsx', sheet_name='DB', dtype=str)
cd_cf11['origen'] = 'CD'

In [8]:
tienda_name = "221121_post"
tienda_cf11 = pd.read_excel(f'input/cierres_f11/{tienda_name}.xlsx', sheet_name='DB', dtype=str)
tienda_cf11['origen'] = 'TIENDA'

In [9]:
cd_to_del = ['indice_cf11','prd_upc', 'transportadora_nuevo', 'reporte_a_contabilidad', 'movimiento_contable',
        'tranf_electro_factura', 'f3', 'f4', 'f5', 'f11_nuevo', 'f12',
       'nota', 'ro', 'gco_dup', 'dupmc', 'error_ru', 'gco_dupall']
cd_cf11.drop(cd_to_del, axis=1, inplace=True)

In [10]:
tienda_to_del = ['indice_c11t', 'ean', 'propietario', 'f']
tienda_cf11.drop(tienda_to_del, axis=1, inplace=True)

In [11]:
tienda_cf11.rename(columns={'folio_servicio_tecnico':'n_folio', 'estado_servicio_tecnico':'estado_f11',
       'costo_promedio':'costo_total', 'tipo_servicio_f11': 'xservicio', 'motivo_cierre':'status_final', 'cantidad_f11':'qproducto'}, inplace=True)

In [12]:
con = pd.concat([cd_cf11, tienda_cf11])
con['partition_date'] = date.today()
con['costo_total'] = pd.to_numeric(con['costo_total'])
con['status_final'] = con.status_final.str.capitalize()

In [13]:
con['Comentario GCO'].unique()

array(['Coincidencia exacta F4+UPC+QTY', 'Coincidencia exacta F3+UPC+QTY',
       'Coincidencia exacta (F12|F11)', 'Coincidencia exacta F5+UPC+QTY',
       nan, 'Duplicidad verificada F12+UPC+Cantidad',
       'F11 nuevo en estado despachado', 'No coincide (F12|F11)',
       'Coincidencia F4+UPC+QTY + Registro duplicado en DB',
       'Duplicidad verificada F11+UPC+Cantidad',
       'Coincidencia F3+UPC+QTY + Registro duplicado en DB',
       'Registro anulado', 'Recibido con fecha posterior al 14/01/2022',
       'No coincide F4|UPC|QTY',
       'Cantidad de las F11s de un F5 > cantidad del F5',
       'Coincidencia exacta (F11|F3)+UPC+QTY',
       'Coincidencia exacta (F11|F4)+UPC+QTY',
       'Coincidencia Exacta (F11|DV)+UPC+QTY',
       'No coincide (F11|F4)+UPC+QTY', 'No coincide (F11|DV)',
       'No coincide (F11|F3)+UPC+QTY',
       'Registro con año de confirmación diferente a 2022',
       'Registro con estado diferente a recibido',
       'Cantidad de los F11s de un F3 > ca

In [14]:
con_nov = [
       'Coincidencia (F12|F11) - Se requiere F12|UPC|QTY para validar duplicidad',
       'F11 Registro sin nro. de f11 nuevo', 
       'No coincide F3|UPC|QTY', 'No coincide (F12|F11)',
       'No coincide F5|UPC|Qty', 'Registro sin nro. de F4',
       'F11 nuevo no encontrado',
       'Recibido con fecha anterior al 14/01/2022', 
       'Recibido con fecha posterior al 14/01/2022', 
       'Registro duplicado F12 + UPC + Cantidad',
       'Coincidencia F4+UPC+QTY + Registro duplicado en DB',
       'Coincidencia (F12|F11) + Registro duplicado en DB',
       'Registro con estado diferente a recibido',
       'Cantidad de los F11s de un F4 > cantidad del F4',
       'No coincide F4|UPC|QTY',
       'Coincidencia F3+UPC+QTY + Registro duplicado en DB',
       'No coincide (F11|F4)+UPC+QTY', 'No coincide (F11|F3)+UPC+QTY',
       'Registro sin nro. de F5',
       'Registro con año de reserva diferente a 2022',
       'La cantidad sumada de los F11s de un F5 es mayor que la cantidad del F5',
       'Registro duplicado F+UPC+Cantidad',
       'Registro con año de confirmación diferente a 2022',
       'Cantidad de los F11s de un F3 > cantidad del F3',
       'Registro anulado','Coincidencia F5+UPC+QTY - Se requiere F12|UPC|QTY para validar duplicidad',
       'Cantidad de las F11s de un F5 > cantidad del F5','Error de clasificacion - Duplicidad F12+UPC+Cantidad', 'F11 nuevo encontrado (diferente estado)', 'Error de clasificacion - duplicidad F11 + upc +cantidad','No coincide (F11|DV)',
       'Coincidencia Exacta (F11|DV) QTY diferente',
       'Coincidencia F4+UPC+QTY - Se requiere F12|UPC|QTY para validar duplicidad', 'Coincidencia F3+UPC+QTY - Se requiere F12|UPC|QTY para validar duplicidad']

sin_nov = ['Coincidencia exacta F4+UPC+QTY', 'Coincidencia exacta F3+UPC+QTY',
'Coincidencia exacta (F12|F11)', 'F11 nuevo en estado despachado',
'Coincidencia exacta F5+UPC+QTY',
'Coincidencia exacta (F11|F3)+UPC+QTY', 'Coincidencia exacta (F11|F4)+UPC+QTY',
'Duplicidad verificada F12+UPC+Cantidad', 'Duplicidad verificada F11+UPC+Cantidad',
 'Error f11', 'Registro de f11 nuevo encontrado','Coincidencia Exacta (F11|DV)+UPC+QTY']

con.loc[con['Comentario GCO'].isin(con_nov), 'rev'] = 'Con novedad'
con.loc[con['Comentario GCO'].isin(sin_nov), 'rev'] = 'Sin novedad'
con['rev'] = con['rev'].fillna('Pendiente')


In [15]:
con.loc[con['rev'].isna(), 'Comentario GCO'].unique()

array([], dtype=object)

In [16]:
con.loc[(con['origen']=='TIENDA') & (con['Comentario GCO'].isna()), 'rev'] = 'Cierre en tienda'

In [17]:
# revision sin novedad
con.loc[(con['rev'] == 'Pendiente') & (con['Comentario GCO'].notna())]['Comentario GCO'].unique() 

array([], dtype=object)

In [18]:
con.loc[(con['rev'] == 'Con novedad') & (con['origen'] == 'CD'),'Comentario GCO'].unique()

array(['No coincide (F12|F11)',
       'Coincidencia F4+UPC+QTY + Registro duplicado en DB',
       'Coincidencia F3+UPC+QTY + Registro duplicado en DB',
       'Registro anulado', 'Recibido con fecha posterior al 14/01/2022',
       'No coincide F4|UPC|QTY',
       'Cantidad de las F11s de un F5 > cantidad del F5'], dtype=object)

In [19]:
con.columns

Index(['nfolio', 'qproducto', 'xobservacion', 'xservicio', 'costo_total',
       'estado_f11', 'status_final', 'sku', 'GCO', 'Comentario GCO', 'checked',
       'fnandup', 'origen', 'n_folio', 'local_envio',
       'sku_(numero_de_producto)', 'F11', 'SKU ORIGINAL',
       'CANT. UND GENERADAS', 'partition_date', 'rev'],
      dtype='object')

In [20]:
# CD
cd_fi = [ 'Escalamiento li',  'Ingreso con sku trocado', 'Pendiente revisar', 'Ingresa con f12 trocado']
cd_error = ['Error en creacion de f11', 'Con nuevo f11 en estado despachado', 'Error de cierre', 'Error en creacia3n de f11']
con.loc[con.status_final.isin(cd_fi), 'status_final'] = 'Falta infomación CD'
con.loc[con.status_final.isin(cd_error), 'status_final'] = 'Error cierre/creación F11'
duplicados = ['Cierre x duplicidad f11+upc+qty', 'Cierre x duplicidad f12+upc+qty']
con.loc[con.status_final.isin(duplicados), 'status_final'] = 'Duplicados'
con.loc[con.status_final=='Cierre x producto guardado despues de inventario', 'status_final'] = 'Guardado después de inv'
con.loc[con.status_final=='Cierre x producto guardado antes de inventario', 'status_final'] = 'Guardado antes de inv'


con.loc[con.status_final=='Cierre x f3 devuelto a proveedor', 'status_final'] = 'F3 devuelto a proveedor'
con.loc[con.status_final=='Cierre x f4 cobrado a terceros', 'status_final'] = 'F4 cobrado a terceros'


In [21]:
# Tienda
tienda_ct = ['Sin informacion - tienda', 'Duplicado con ajuste en tienda',
       'Entregado en tienda', 'Cambio producto x nuevo', 'Dv','Ajuste inventarios', 
       'Entregado a cliente', 'Validar cierre con chile', 'F11 concesion', 'Nuevo f11']
con.loc[con.status_final.isin(tienda_ct), 'status_final'] = 'Cierre en tienda'
con.loc[con.status_final=='F3', 'status_final'] = 'F3 devuelto a proveedor'
con.loc[con.status_final=='F4 en tienda', 'status_final'] = 'F4 de merma'
con.loc[con.status_final=='F4', 'status_final'] = 'F4 de merma'

In [22]:
con['status_final'].replace('Cierre por producto guardado despues de inventario','Guardado despues de inv', inplace= True)
con['status_final'].replace('Cierre por f4 cobrado a terceros','F4 cobrado a terceros', inplace= True)
con['status_final'].replace('Cierre por f3 devuelto a proveedor','F3', inplace= True)
con['status_final'].replace('F4 de merma','F4 merma', inplace= True)
con['status_final'].replace('F3 devuelto a proveedor','F3', inplace= True)

In [23]:
con_total = con.groupby('status_final')['costo_total'].sum().sort_values(ascending=False).reset_index()

In [24]:
lista = con_total['status_final'][0:6].tolist()

In [25]:
lista

['F3',
 'Guardado despues de inv',
 'Cierre en tienda',
 'F4 cobrado a terceros',
 'F4 merma',
 'Duplicados']

In [26]:
con_graf = con_total.loc[con_total['status_final'].isin(lista)]

In [27]:
total_f11s = con_total['costo_total'].sum()

In [28]:
fig_con_total = px.bar(con_graf, x ='status_final', y='costo_total', title=f"Costo total de F11s según motivos de cierre - $ {total_f11s/1e6:,.0f}M", labels={ 'costo_total':'Total costo promedio', 'status_final':'Motivo de cierre', 'GRUPO':'Local'}, text='costo_total',text_auto='.2s' ) #category_orders=co ) 
#fig_con_total.update_layout(legend=dict(yanchor="top", y=0.9, xanchor="left", x=0.1))
fig_con_total.write_image(f'output/{dt_string}_con_total.svg',width=1050, height=300, engine='orca') # 820 , 350 
fig_con_total.show()

In [29]:
con.loc[(con.origen =='CD') & (con['rev'] == 'Pendiente'), 'status_final'] = 'Falta infomación CD'

In [30]:
con_cd = con.loc[con.origen =='CD'].groupby(['status_final', 'rev'], sort=False)['costo_total'].sum().sort_values(ascending=False).reset_index()

In [31]:
motivos = con_cd.status_final.unique()
gb_motivos_cd = con_cd.groupby('status_final')['costo_total'].sum().reset_index()
gb_motivos_cd.set_index('status_final', inplace=True)
for motivo in motivos:
    con_cd.loc[con_cd['status_final']==motivo, 'status_final'] = f'{motivo} - ${round(gb_motivos_cd.loc[motivo, "costo_total"]/1e6):,.0f}M'

In [32]:
revs = con_cd.rev.unique()
gb_rev_cd = con_cd.groupby('rev')['costo_total'].sum().reset_index()
gb_rev_cd.set_index('rev', inplace=True)
for rev in revs:
    con_cd.loc[con_cd['rev']==rev, 'rev'] = f'{rev} - ${round(gb_rev_cd.loc[rev, "costo_total"]/1e6):,.0f}M'

In [33]:
revs =  [con_cd.loc[con_cd['rev'].str.contains(r'Sin novedad'), 'rev'].unique()[0], con_cd.loc[con_cd['rev'].str.contains(r'Con novedad'), 'rev'].unique()[0], con_cd.loc[con_cd['rev'].str.contains(r'Pendiente'), 'rev'].unique()[0]]
co_cd = {'rev': revs, 'status_final':con_cd.status_final.unique()}

In [34]:
total_cd= con_cd['costo_total'].sum()

In [35]:
fig_con_total = px.bar(con_cd, orientation='h', y ='status_final', x='costo_total', color='rev', category_orders=co_cd, title= f"Motivos de cierres de F11 CD 2022 - $ {total_cd/1e6:,.0f}M", 
labels={ 'costo_total':'Total costo promedio', 'status_final':'Motivo de cierre', 'rev':'Estado revisión'}, text='costo_total',text_auto='.2s', 
color_discrete_sequence=[ 'rgb(115, 175, 72)','rgb(204, 80, 62)','rgb(179, 179, 179)' ] ) 
fig_con_total.update_layout(legend=dict(yanchor="bottom", y=0.1, xanchor="right", x=0.9))
fig_con_total.update_layout(margin_r=20, margin_t=60)
fig_con_total.update_xaxes(range = [0,con_cd['costo_total'].max() + con_cd['costo_total'].max() * 0.2], constrain = 'domain')
fig_con_total.write_image(f'output/{dt_string}_con_cd.svg',width=1000, height=400, engine='orca') # 820 , 350 
fig_con_total.show()

In [36]:
con_tienda = con.loc[con.origen =='TIENDA'].groupby(['status_final', 'rev'])['costo_total'].sum().sort_values(ascending=False).reset_index()

In [37]:
con_tienda

,status_final,rev,costo_total
0,F3,Sin novedad,4.533410e+09
1,Cierre en tienda,Cierre en tienda,1.923061e+09
2,Cierre en tienda,Sin novedad,1.361262e+09
3,F4 merma,Sin novedad,3.653966e+08
4,F5,Sin novedad,1.710918e+08
5,F3,Con novedad,3.511312e+07
6,Cierre en tienda,Con novedad,2.941116e+07
7,F4 merma,Con novedad,2.581822e+07
8,F5,Con novedad,1.429123e+06


In [38]:
motivos = con_tienda.status_final.unique()
gb_motivos_tienda = con_tienda.groupby('status_final')['costo_total'].sum().reset_index()
gb_motivos_tienda.set_index('status_final', inplace=True)
for motivo in motivos:
    con_tienda.loc[con_tienda['status_final']==motivo, 'status_final'] = f'{motivo} - ${round(gb_motivos_tienda.loc[motivo, "costo_total"]/1e6):,.0f}M'

In [39]:
revs = con_tienda.rev.unique()
gb_rev_tienda = con_tienda.groupby('rev')['costo_total'].sum().reset_index()
gb_rev_tienda.set_index('rev', inplace=True)
for rev in revs:
    con_tienda.loc[con_tienda['rev']==rev, 'rev'] = f'{rev} - ${round(gb_rev_tienda.loc[rev, "costo_total"]/1e6):,.0f}M'

In [40]:
revs

array(['Sin novedad', 'Cierre en tienda', 'Con novedad'], dtype=object)

In [41]:
con_tienda.loc[con_tienda['rev'].str.contains(r'Sin novedad'), 'rev'].unique()[0], con_tienda.loc[con_tienda['rev'].str.contains(r'Con novedad'), 'rev'].unique()[0], con_tienda.loc[con_tienda['rev'].str.contains(r'Cierre en tienda'), 'rev'].unique()[0]

('Sin novedad - $6,431M', 'Con novedad - $92M', 'Cierre en tienda - $1,923M')

In [42]:
revs_tienda =  [con_tienda.loc[con_tienda['rev'].str.contains(r'Sin novedad'), 'rev'].unique()[0], con_tienda.loc[con_tienda['rev'].str.contains(r'Con novedad'), 'rev'].unique()[0], con_tienda.loc[con_tienda['rev'].str.contains(r'Cierre en tienda'), 'rev'].unique()[0]]
co_tienda = {'rev':revs_tienda, 'status_final':con_tienda.status_final.unique()}

In [43]:
total_tienda = con_tienda['costo_total'].sum()

In [44]:
fig_con_total = px.bar(con_tienda, orientation='h', y ='status_final', x='costo_total', title= f"Motivos de cierres de F11 Tienda 2022 - $ {total_tienda/1e6:,.0f}M", 
labels={ 'costo_total':'Total costo promedio', 'status_final':'Motivo de cierre', 'rev':'Estado revisión'}, text='costo_total',text_auto='.2s', 
color_discrete_sequence=[ 'rgb(115, 175, 72)','rgb(204, 80, 62)','rgb(179, 179, 179)' ], color ='rev', category_orders=co_tienda ) 
fig_con_total.update_layout(legend=dict(yanchor="bottom", y=0.1, xanchor="right", x=0.9))
fig_con_total.update_layout(margin_r=20, margin_t=60)
fig_con_total.update_xaxes(range = [0,con_tienda['costo_total'].max() + con_tienda['costo_total'].max() * 0.2], constrain = 'domain')
fig_con_total.write_image(f'output/{dt_string}_con_tienda.svg',width=900, height=400, engine='orca') # 820 , 350 
fig_con_total.show()

In [45]:
con.head()

,nfolio,qproducto,xobservacion,xservicio,costo_total,estado_f11,status_final,sku,GCO,Comentario GCO,checked,fnandup,origen,n_folio,local_envio,sku_(numero_de_producto),F11,SKU ORIGINAL,CANT. UND GENERADAS,partition_date,rev
0,1111598080,1,se cierra 1111598080 asociado a 12979073643 p...,retiro f12,344000.0,entrega total,F4 cobrado a terceros,4094654,OKK,Coincidencia exacta F4+UPC+QTY,y,NaN,CD,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-21,Sin novedad
1,1111773746,1,asociado a f3 33522586,retiro f12,6230.0,entrega total,F3,6208803,OKK,Coincidencia exacta F3+UPC+QTY,y,NaN,CD,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-21,Sin novedad
2,1111771537,1,asociado a f3 33522548,retiro f12,35450.0,entrega total,F3,22286520,OKK,Coincidencia exacta F3+UPC+QTY,y,NaN,CD,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-21,Sin novedad
3,1111774354,1,asociado a f3 33547015,retiro f12,93990.0,entrega total,F3,33369159,OKK,Coincidencia exacta F3+UPC+QTY,y,NaN,CD,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-21,Sin novedad
4,1111236937,1,f1285237808 cliente informa quese cierra 11112...,retiro f12,11880.0,entrega total,F4 merma,770231173,OKK,Coincidencia exacta F4+UPC+QTY,y,NaN,CD,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-21,Sin novedad


In [46]:
con.to_excel('consolidado_cf11.xlsx', index=False)